# Cleaning Trial

In [206]:
import pandas as pd
import numpy as np
import re

In [218]:
constituency = '070'
folder = '/home/hennes/Internship/constituencies/'
df = pd.read_csv(f'{folder}AC{constituency}.csv')

In [219]:
# make F nan

repl_dict = {'F': np.NaN} 

df = df.replace(repl_dict, regex=True)

In [220]:
# delete columns that have only have a few values in them. They are most likely useless.
df.dropna(thresh=len(df) - (len(df)/2), axis=1, inplace=True)
# delete rows that have more than 5 missing values
df.dropna(thresh = (len(df.columns)/1.4), axis = 0, inplace = True)
df.reset_index(drop=True, inplace=True)

Rectifying systematic errors

In [221]:
# transform systematic errors
repl_dict = {'\$':'5',
            '\(4\)':'(A)',
            '4\)':'(A)',
            '(\(A\))|(A\))|(\(A)|A':'A',
            '(\.0)$':'',
            'v':'0',
            '_':'',
            '\]':'',
            '\[':'',
            '\|':'',
            '\.':'',
            '[\(\)]':'',
            ' ': '',
            '(?!A)\D':'',
            '^\s*$':np.NaN} 

df = df.replace(repl_dict, regex=True)

In [222]:
# replace values with 4 or more digits with NAN

repl_dict = {'\d{4,}': np.NaN} 

df = df.replace(repl_dict, regex=True)
df = df.replace(r'\s+( +\.)|#',np.nan,regex=True).replace('',np.nan)

In [223]:
# delete rows that have more than 5 missing values
df.dropna(thresh = (len(df.columns)-3), axis = 0, inplace = True)
df.reset_index(drop=True, inplace=True)

Naming columns

In [224]:
# The two columns with the highest numbers should be total valid votes and total votes.
# Total valid votes is to the left of total votes.

# first need to convert columns to int. Should only do that with the non-serial number columns.
# This mask selects all columns that do not have 'A' in them.
mask = df[[e for e in df.columns]].apply(lambda x:
                                         x.astype(str).str.contains(r'A', regex=True)).any(axis='index')

# The first is all columns except serial number, second on is only serial number
serial = [df.iloc[:,2].name]
not_serial = df.loc[:,df.columns != serial[0]].columns.tolist()

# then convert all remaining characters to numeric or nan
for col in not_serial:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col] = df[col].astype(float)

# name column with highest median value 'total votes'    
df.rename(columns = {df.median().idxmax(axis=1):'total'}, inplace=True)

# exclude it and name the one with second highest median 'total_valid'
columns = [col for col in df.columns if not col.startswith('total')]
df.rename(columns = {df[columns].median().idxmax(axis=1):'total_valid'}, inplace=True)

# sometimes total and total valid will be switched.
# Check if column three places to left of total_valid is called total
# If so, switch their names
if df.columns.tolist()[df.columns.tolist().index('total_valid')-3] == 'total':
    df.rename(columns = {'total': 'total_valid', 'total_valid':'total'}, inplace=True)

In [225]:
# get names of columns one and two places to right of 'total valid'
sublist = ['total_valid']
rejected = df.columns.tolist()[(df.columns.get_indexer(sublist)+1)[0]]
nota = df.columns.tolist()[(df.columns.get_indexer(sublist)+2)[0]]
first = df.columns.tolist()[0]
second = df.columns.tolist()[1]

# rename first column 'page_idx' and other columns
df.rename(columns={rejected:'rejected',
                   nota:'nota',
                   first:'page_idx',
                   second:'serial_1',
                  serial[0]:'serial'}, inplace = True)

In [226]:
# delete all rows in which no cell has more than two digits (also accounting for .0)

mask = df.apply(lambda x: x.astype(str).str.contains(r'^\d{,2}(\.0)?$', regex=True)).all(axis=1)
df = df[~mask]
df.reset_index(drop=True, inplace=True)

/home/hennes/.local/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [227]:
# correcting columns wrongly shifted

# Identify all rows that have NAN in the column furthest right
# and which do not have an 'A' in the serial column. Those should not be moved
rowlist = df[(df.iloc[:,-1].isna()) & (~df['serial'].str.contains('A', na=False))].index.tolist()

# Calculate how many standard deviations all values of each row are away from the average of the respective columns 
for row in rowlist:
    collist = df.dtypes[df.dtypes == float].index.tolist()
    sdlist_old = []
    for col in collist:
        sdlist_old.append((((df.loc[row, col] - df[col].mean())**2)**0.5) / df[col].std())
    
# Compute the average standard deviation for each of these rows 
    rowsd_old = np.nanmedian(sdlist_old)

# Shift the values of the row to the right and report the new average standard deviation 
    df1 = df.copy(deep=True)
    df1.loc[row, :] = df1.loc[row, :].shift(1, axis=0)
    collist = df1.dtypes[df1.dtypes == float].index.tolist()
    sdlist_new = []
    for col in collist:
        sdlist_new.append((((df1.loc[row, col] - df1[col].mean())**2)**0.5) / df1[col].std())
    
    rowsd_new = np.nanmedian(sdlist_new)
    
# Take over the shift if the new SD is smaller than the old SD

    if rowsd_old > rowsd_new:
        df.loc[row] = df1.loc[row]
        print(f'Shifted row {row} to the right.')
        
        serial = [df.iloc[:,2].name]
        not_serial = df.loc[:,df.columns != serial[0]].columns.tolist()

        # then convert all remaining characters to numeric or nan
        for col in not_serial:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col] = df[col].astype(float)

Shifted row 188 to the right.
Shifted row 356 to the right.
Shifted row 357 to the right.
Shifted row 358 to the right.


## Correcting First Serial Numbers

In [228]:
# For errors without a gap (no row is missing)

# Idea is: if (n)+1 does not equal (n+1), then see if (n-1)+2 equals (n+1)
# This logic is extended for up to 5 numbers ahead of n. In this way, gaps 
# of up to 4 numbers will be bridged. At the same time, there will be no
# interpolation if there is no clean continuation of integers.

for n in df.index.tolist()[1:]:
    try:
        if df.iloc[n,1] != df.iloc[n-1,1]+1:
            if df.iloc[n-1,1]+2 == df.iloc[n+1,1]:
                df.iloc[n,1] = df.iloc[n-1,1]+1
            if df.iloc[n-1,1]+3 == df.iloc[n+2,1]:
                df.iloc[n,1] = df.iloc[n-1,1]+1
                df.iloc[n+1,1] = df.iloc[n-1,1]+2
            if df.iloc[n-1,1]+4 == df.iloc[n+3,1]:
                df.iloc[n,1] = df.iloc[n-1,1]+1
                df.iloc[n+1,1] = df.iloc[n-1,1]+2
                df.iloc[n+2,1] = df.iloc[n-1,1]+3
            if df.iloc[n-1,1]+5 == df.iloc[n+4,1]:
                df.iloc[n,1] = df.iloc[n-1,1]+1
                df.iloc[n+1,1] = df.iloc[n-1,1]+2
                df.iloc[n+2,1] = df.iloc[n-1,1]+3
                df.iloc[n+3,1] = df.iloc[n-1,1]+4
            if df.iloc[n-1,1]+6 == df.iloc[n+5,1]:
                df.iloc[n,1] = df.iloc[n-1,1]+1
                df.iloc[n+1,1] = df.iloc[n-1,1]+2
                df.iloc[n+2,1] = df.iloc[n-1,1]+3
                df.iloc[n+3,1] = df.iloc[n-1,1]+4
                df.iloc[n+4,1] = df.iloc[n-1,1]+5
    except:
        None
        
# For errors with a gap (occur often between last row of one page and first row of next)
# define index of last row of each page
last = [x-1 for x in df[df['page_idx']==0].index.tolist()[1:]]

# if (n) does not equal (n-1)+1, then make it so
for n in last:
    try:
        if df.iloc[n,1] != df.iloc[n-1,1]+1:
            df.iloc[n,1] = df.iloc[n-1,1]+1
    except:
        None

# for errors at beginning of pages

for n in df[df['page_idx']==0].index.tolist()[1:]:
    if df.iloc[n,1] != df.iloc[n+1,1]-1:
        df.iloc[n,1] = df.iloc[n+1,1]-1


before: 359.0, nan
after: 359.0, 360.0


In [229]:
# for errors at beginning of pages

for n in df[df['page_idx']==0].index.tolist()[1:]:
    if df.iloc[n,1] != df.iloc[n+1,1]-1:
        df.iloc[n,1] = df.iloc[n+1,1]-1


before: nan, 1.0
after: 0.0, 1.0


In [230]:
df

,page_idx,serial_1,serial,2,3,4,5,6,7,total_valid,rejected,nota,total,12
0,0.0,1.0,1,497.00,156.0,135.0,5.0,2.0,0.0,795.0,0.0,17.0,812.0,0.0
1,1.0,2.0,2,42.20,100.0,132.0,3.0,7.0,1.0,665.0,0.0,14.0,679.0,0.0
2,2.0,3.0,3,396.00,74.0,271.0,2.0,4.0,0.0,747.0,0.0,5.0,752.0,0.0
3,3.0,4.0,4,319.00,26.0,122.0,0.0,0.0,0.0,467.0,0.0,1.0,468.0,0.0
4,4.0,5.0,5,440.00,96.0,206.0,2.0,3.0,1.0,748.0,0.0,5.0,753.0,0.0
5,5.0,6.0,6,445.00,30.0,231.0,2.0,4.0,3.0,715.0,0.0,9.0,724.0,0.0
6,6.0,7.0,7,500.00,105.0,67.0,1.0,3.0,2.0,678.0,0.0,12.0,690.0,0.0
7,7.0,8.0,8,248.00,83.0,18.0,2.0,1.0,3.0,355.0,0.0,9.0,364.0,0.0
8,8.0,9.0,9,2.08,80.0,421.0,6.0,6.0,2.0,723.0,0.0,8.0,731.0,0.0
9,9.0,10.0,10,3.00,138.0,205.0,5.0,0.0,1.0,352.0,0.0,2.0,354.0,0.0


In [231]:
# trying out rule to take care of numbers missing digits

for n in [e for e in df.index.tolist()[1:] if e not in df[df['page_idx']==0].index.tolist()[1:]]:
    try:
        if df.iloc[n,1] != df.iloc[n-1,1]+1:
            print(f'before: {df.iloc[n-2,1]} first: {df.iloc[n-1,1]} second:{df.iloc[n,1]}')
    except:
        None

## Renaming Columns according to Candidates

### Getting rank-party pairs

In [233]:
# import excel on candidate names
d = pd.read_excel('/home/hennes/Internship/Candidate_Name_Party_Data.xlsx')
# define df excluding NOTA, only current constituency
dat = d[(d['PARTY']!= 'NOTA')
        & (d['AC NO.']== float(re.findall(r'[1-9][0-9]*',f'AC{constituency}.csv')[0]))][['AC NO.', 'PARTY', 'TOTAL']]
# get number of candidates
n_candidates = len(dat)

In [234]:
# create column with rank of party per constituency
dat['rank'] = dat.groupby('AC NO.').rank(ascending=False)
# create dict with party value pair
rank_party = pd.Series(dat.PARTY.values,index=dat['rank']).to_dict()
rank_party

{1.0: 'AITC', 2.0: 'BJP', 3.0: 'INC', 4.0: 'SUCI', 5.0: 'IUML', 6.0: 'HAMS'}

### Getting column-rank pairs

In [235]:
# create dictionary with key = column name, value = rank
serial = df.columns.get_indexer(['serial'])[0]
column_rank = df.iloc[:,serial+1:serial+(n_candidates+1)]\
    .agg(func=np.sum)\
    .rank(ascending=False)\
    .to_dict()

### Renaming column according to rank

In [236]:
rename_dict={}
for col, rank in column_rank.items():
    rename_dict.update({col:rank_party.get(rank)}) 

df.rename(columns=rename_dict, inplace=True)

In [237]:
################ create constituency number column #######################
    
df['ac'] = re.findall(r'[1-9][0-9]*' ,f'AC{constituency}.csv')[0]

## Some methods to apply

In [204]:
# select all rows with specific regex values

mask = df.apply(lambda x: x.astype(str).str.contains(r'^\d{,2}(.0)?$', regex=True)).all(axis=1)
df[mask]

,page_idx,0,serial,2,3,4,5,6,total_valid,rejected,nota,total,11
0,0.0,1.0,2,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0


In [182]:
# select all columns with specific regex values

mask = df[[e for e in df.columns]].apply(lambda x: x.astype(str).str.contains(r'((?!A)\D)', regex=True)).any(axis=1)
df[mask]

/home/hennes/.local/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0,1,1,9,156.0,66,213,5,5,2,456,0,7,463,0
1,1,2,1A,3,152.0,78,249,4,0,2,488,0,,495,0
2,2,3,2,10,250.0,76,326,8,1,2,673,0,8,681,0
3,3,4,3,5,170.0,61,177,3,2,0,418,0,4,422,0
4,4,5,4,9,287.0,54,354,6,1,5,716,0,9,725,0
5,5,6,5,0,183.0,75,223,3,0,1,485,9,2,487,0
6,6,7,6,3,238.0,124,332,11,4,2,716,0,6,722,0
7,7,8,7,1,150.0,1,160,3,0,3,418,0,2,420,0
8,8,9,7A,5,111.0,56,242,0,1,1,416,0,0,416,0
9,9,10,8,9,222.0,70,352,8,2,4,667,0,13,680,0
